In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%pip install -q amplpy matplotlib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 10.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("sp500_2003_2023.csv")

In [5]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index("date")

In [6]:
sample = df.sample(20, axis=1)

In [169]:
corr = sample.corr().stack()
corr[corr.abs() < 0.8].index

MultiIndex([('DRI.US',  'STT.US'),
            ('DRI.US',  'APA.US'),
            ('DRI.US',  'PPL.US'),
            ('UNH.US',  'STT.US'),
            ('UNH.US',  'APA.US'),
            ('UNH.US',  'PPL.US'),
            ('COR.US',  'STT.US'),
            ('COR.US',  'APA.US'),
            ('COR.US',  'PPL.US'),
            ('COR.US',  'AMD.US'),
            ...
            ('AMD.US',  'STT.US'),
            ('AMD.US',  'APA.US'),
            ('AMD.US',  'PPL.US'),
            ('AMD.US', 'NTRS.US'),
            ('HPQ.US',  'COR.US'),
            ('HPQ.US', 'AMGN.US'),
            ('HPQ.US',  'STT.US'),
            ('HPQ.US',  'APA.US'),
            ('HPQ.US',  'PPL.US'),
            ('HPQ.US', 'EBAY.US')],
           length=108)

In [8]:
corr

NTRS.US  NTRS.US    1.000000
         MET.US     0.848834
         MMM.US     0.867924
         SNPS.US    0.733046
         ETN.US     0.792203
                      ...   
ECL.US   GEN.US     0.865186
         PG.US      0.924638
         ABT.US     0.940535
         XOM.US     0.565909
         ECL.US     1.000000
Length: 400, dtype: float64

In [13]:
corr = sample.corr().stack()
corr = corr[corr < 1]
corr_subset = corr[corr.abs() > 0.8].index
keep, drop = set(), set()
for s1, s2 in corr_subset:
  if s1 not in keep:
    if s2 not in keep:
      keep.add(s1)
      drop.add(s2)
    else:
      drop.add(s1)
  else:
    keep.discard(s2)
    drop.add(s2)





In [18]:
corr[corr.abs() > 0.8]

NTRS.US  MET.US    0.848834
         MMM.US    0.867924
         NOC.US    0.895202
         HAS.US    0.903979
         AFL.US    0.854744
                     ...   
ECL.US   AVY.US    0.868053
         PPG.US    0.961991
         GEN.US    0.865186
         PG.US     0.924638
         ABT.US    0.940535
Length: 264, dtype: float64

In [16]:
drop.intersection(keep)

{'ABT.US',
 'AFL.US',
 'AMD.US',
 'AVY.US',
 'D.US',
 'ETN.US',
 'GEN.US',
 'HAS.US',
 'HSY.US',
 'MET.US',
 'MMM.US',
 'NOC.US',
 'PG.US',
 'PPG.US',
 'RTX.US',
 'SNPS.US',
 'WY.US'}

In [11]:
len(keep)

19

In [159]:
returns = sample.pct_change().dropna()

In [ ]:
returns = {"A1": 0.0476,
           "A2": 0.004}

covariances = {("A1", "A1"): 2.12,
               ("A1", "A2"): 1.03,
               ("A2", "A1"): 1.03,
               ("A2", "A2"): 1.89}


In [ ]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin"],
    license_uuid="default"
)

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [ ]:
%%ampl_eval
#define sets
reset;

set ASSETS;

param returns {ASSETS};
param covars {ASSETS, ASSETS} >= 0;
param rfr >= 0, <= 1;

var f{a in ASSETS} >= 0, <= 1;

maximize Profit:
  rfr + sum {a in ASSETS} f[a] * (returns[a] - rfr) -
  0.5 * sum {a in ASSETS} sum {s in ASSETS} covars[a,s] * f[a] * f[s];

subject to WeightConstraint:
  sum {a in ASSETS} f[a] == 1;

In [ ]:
ASSETS = returns.keys()
rfr = 0

ampl.set['ASSETS'] = ASSETS
ampl.param['returns'] = returns
ampl.param['covars'] = covariances
ampl.param['rfr'] = rfr

ampl.option["solver"] = "ipopt"
ampl.solve()

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Tot

In [ ]:
print(f"Profit = {ampl.obj['Profit'].value()}")

print("\n Allocation Report")
for asset, f in ampl.var["f"].to_dict().items():
  print(f" {asset} = {f}")

Profit = -0.7316428307692309

 Allocation Report
 A1 = 0.46338461615155424
 A2 = 0.5366153838484458


In [ ]:
def optFraction(f, returns, varcov):
  return np.asarray(f.dot(returns) - f.dot(varcov).dot(f) / 2)

In [ ]:
risk_frac1 = np.linspace(0, 1, 1000)
risk_frac2 = 1 - risk_frac1
F = np.hstack([risk_frac1.reshape(-1, 1),
               risk_frac2.reshape(-1, 1)])

returns = np.array([0.0476, 0.004]) # mu
varcov = np.matrix([[2.12, 1.03], # sigma_ij
                    [1.03, 1.89]])
rfr = 0

g = []
for idx, f in enumerate(F):
  g.append(optFraction(f, returns, varcov))

g = np.vstack(g)
max_pt = np.argmax(g)
print(f"Max fractions = {F[max_pt].round(3)}\ng = {g.max():.3f}")

Max fractions = [0.463 0.537]
g = -0.732


In [ ]:
g.max()

-0.7316428368308248